# Experiments with the Housing dataset

## Intro

The housing dataset has many ordinal features and several categorical ones. In order to work with least squares method we mapped ordinal features as specified in the following list (the numeric value assigned to the ordinal value names is the position in the list associated with the feature name, starting from zero). 

- LotArea: possibile serva un 1hot encoding
- Street: si può determinare un ordine
- Alley: si può determinare un ordine (N/A peggiore)
- LotShape: IR3 - IR2 - IR1 - Reg
- LandContour: 1 hot encoding 
- LotConfig: 1 hot encoding
- LandSlope: Sev - Mod - Gtl
- Utilities: ELO - NoSeWa - NoSewr - AllPub
- Neighborhood: 1hot
- Condition1: 1 hot
- Condition2: 1 hot
- BldgType: 1Fam, 1FmCon, TwnhsI, TwhsE, Duplx
- HouseStyle: 1Story,  1.5Unf,  1.5Fin,  2Story,  2.5Unf,  2.5Fin,  SFoyer,  SLvl
- RoofStyle: 1 hot 
- RoofMatl: 1 hot
- Ext1,2: 1 hot
- MasVnrType: 1 hot
- ExterQual: Po, Fa, TA, Gd, Ex,
- ExterCond: Po, Fa, TA, Gd, Ex
- Foundation: Wood, Stone, Slab, PConc, CBlock, BrkTil
- Functional: Sal, Sev, Maj2, Maj1, Mod, Min2, Min1, Typ
- BsmtQual: NA, Po, Fa, TA, Gd, Ex
- BsmtCond: NA, Po, Fa, TA, Gd, Ex
- BsmtExposure: NA, No, Mn, Av, Gd
- OverallQual: 1 ~ 10
- BsmtFinType1: NA, Unf, LwQ, Rec, BLQ, ALQ, GLQ
- BsmtFinType2: see above
- Heating: onehot
- Exterior 1st: onehot
- Exterior 2nd: onehot
- MasVnrType: onehot
- HeatingQC: Po, Fa, TA, Gd, Ex
- CentralAir: N, Y
- Electrical: onehot
- KitchenQual: Po, Fa, TA, Gd, Ex
- FireplaceQu: NA, Po, Fa, TA, Gd, Ex
- GarageType: NA, Detchd, CarPort, BuiltIn, Basment, Attchd, 2Types
- GarageFinish: NA, Unf, RFn, Fin
- GarageQual: NA, Po, Fa, TA, Gd, Ex
- GarageCond: NA, Po, Fa, TA, Gd, Ex
- PavedDrive: N, P, Y
- PoolQC: NA, Fa, TA, Gd, Ex
- Fence: NA, MnWw, GdWo, MnPrv, GdPrv
- MiscFeature: onehot
- Saletype: onehot
- SaleCondition: onehot


In [4]:
using Pkg
Pkg.activate(".")

using DataFrames
using CSV
using PartitionedLS
using Statistics
using Logging
using MLJ
using LinearAlgebra

include("exps_helpers.jl")

  Activating new project at `~/UNITO/Research/PartitionedLeastSquares/Julia/PartitionedLS-experiments-2/housing/housing`


ArgumentError: ArgumentError: Package CSV not found in current path, maybe you meant `import/using .CSV`.
- Otherwise, run `import Pkg; Pkg.add("CSV")` to install the CSV package.

## Data Loading and preprocessing

In [6]:
# Note: test data does not have SalePrice... ???
cols = ["MSSubClass", "MSZoning", "LotFrontage", "LotArea", "Street", "Alley", "LotShape", "LandContour", "LotConfig", "LandSlope", "Utilities", "Neighborhood", "Condition1", "Condition2", "YearBuilt", "YearRemodAdd", "BldgType", "HouseStyle", "OverallQual", "OverallCond", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "MasVnrArea", "ExterQual", "ExterCond", "Foundation", "Functional", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "Heating", "HeatingQC", "CentralAir", "Electrical", "Fireplaces", "FireplaceQu", "1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea", "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr", "KitchenQual", "TotRmsAbvGrd", "GarageType", "GarageYrBlt", "GarageFinish", "GarageCars", "GarageArea", "GarageQual", "GarageCond", "PavedDrive", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "PoolArea", "PoolQC", "Fence", "MiscFeature", "MiscVal", "MoSold", "YrSold", "SaleType", "SaleCondition"];
data = CSV.File("train.csv") |> DataFrame;

Pdf = CSV.File("P.csv") |> DataFrame;
Xdf = data[!, cols]
y = data[!, "SalePrice"];

In [7]:
columns = names(Pdf)[2:end]

for col in columns
    print(col)
    print(": ")
    println(join(filter(col => x -> x == 1, Pdf).Descriptor, ","))
end



# filter(:LotDescritption => x -> x == 1, Pdf).Descriptor

LotDescritption: MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope
BuildingPlacement: Utilities,Neighborhood,Condition1,Condition2
BuildingAge: YearBuilt,YearRemodAdd
BuildingQuality: BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,Functional
Basement: BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
PowerAndTemperature: Heating,HeatingQC,CentralAir,Electrical,Fireplaces,FireplaceQu
Sizes: 1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea
Rooms: BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd
OutsideFacilities: GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence
Various: MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition


## Processing ordinal and categorical features

In [8]:
Xdf_clean = preprocess_ordinal_features!(Xdf)
size(Xdf_clean), size(Pdf)

((1460, 79), (79, 11))

In [10]:
Xdf_clean = preprocess_categorical_features(Xdf_clean)
first(Xdf_clean, 5)

Row,LotFrontage,LotArea,Street,Alley,LotShape,LandSlope,Utilities,YearBuilt,YearRemodAdd,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,Foundation,Functional,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,CentralAir,Fireplaces,FireplaceQu,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,MoSold,YrSold,LandContour_Lvl,LandContour_Bnk,LandContour_Low,LandContour_HLS,LotConfig_Inside,LotConfig_FR2,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR3,Neighborhood_CollgCr,Neighborhood_Veenker,Neighborhood_Crawfor,Neighborhood_NoRidge,Neighborhood_Mitchel,Neighborhood_Somerst,Neighborhood_NWAmes,Neighborhood_OldTown,Neighborhood_BrkSide,Neighborhood_Sawyer,Neighborhood_NridgHt,Neighborhood_NAmes,Neighborhood_SawyerW,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Edwards,Neighborhood_Timber,Neighborhood_Gilbert,Neighborhood_StoneBr,Neighborhood_ClearCr,Neighborhood_NPkVill,Neighborhood_Blmngtn,Neighborhood_BrDale,Neighborhood_SWISU,Neighborhood_Blueste,Condition1_Norm,Condition1_Feedr,Condition1_PosN,Condition1_Artery,Condition1_RRAe,Condition1_RRNn,⋯
,String3,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,65,8450,1,0,3,2,3,2003,2003,7,5,196,3,2,3,7,4,3,1,6,706,1,0,150,856,4,1,0,0,856,854,0,1710,1,0,2,1,3,1,3,8,5,2003,2,2,548,3,3,2,0,61,0,0,0,0,0,0,0,2,2008,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,⋯
2,80,9600,1,0,3,2,3,1976,1976,6,8,0,2,2,4,7,4,3,4,5,978,1,0,284,1262,4,1,1,3,1262,0,0,1262,0,1,2,0,3,1,2,6,5,1976,2,2,460,3,3,2,298,0,0,0,0,0,0,0,0,5,2007,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,⋯
3,68,11250,1,0,2,2,3,2001,2002,7,5,162,3,2,3,7,4,3,2,6,486,1,0,434,920,4,1,1,3,920,866,0,1786,1,0,2,1,3,1,3,6,5,2001,2,2,608,3,3,2,0,42,0,0,0,0,0,0,0,9,2008,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,⋯
4,60,9550,1,0,2,2,3,1915,1970,7,5,0,2,2,5,7,3,4,1,5,216,1,0,540,756,3,1,1,4,961,756,0,1717,1,0,1,0,3,1,3,7,1,1998,1,3,642,3,3,2,0,35,272,0,0,0,0,0,0,2,2006,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,⋯
5,84,14260,1,0,2,2,3,2000,2000,8,5,350,3,2,3,7,4,3,3,6,655,1,0,490,1145,4,1,1,3,1145,1053,0,2198,1,0,2,1,4,1,3,9,5,2000,2,3,836,3,3,2,192,84,0,0,0,0,0,0,0,12,2008,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,⋯


## Converting features to type Float64

In [11]:
typesToColumns = mapTypesToColumns(Xdf_clean)

Dict{DataType, Vector{Symbol}} with 3 entries:
  Int64   => [:LotArea, :Street, :Alley, :LotShape, :LandSlope, :Utilities, :Ye…
  String7 => [:MasVnrArea, :GarageYrBlt]
  String3 => [:LotFrontage]

In [12]:
# Ints to Floats
Xdf_clean[!, typesToColumns[Int64]] = convert.(Float64, Xdf_clean[!, typesToColumns[Int64]]);

In [13]:

for column in typesToColumns[String7]
    Xdf_clean[!, column] .= parseStrings(String7, Xdf_clean[!, column])
end


In [14]:

for column in typesToColumns[String3]
    Xdf_clean[!, column] .= parseStrings(String3, Xdf_clean[!, column])
end


In [15]:
# Sanity check: if everything is ok, the following should result in a map with a single key, Float64
mapTypesToColumns(Xdf_clean)

Dict{DataType, Vector{Symbol}} with 1 entry:
  Float64 => [:LotFrontage, :LotArea, :Street, :Alley, :LotShape, :LandSlope, :…

## Updating the P matrix to include new one hot feautres

In [16]:
# process P.csv and update it depending on the onehot encoding
feature_partitions = CSV.read("P.csv", DataFrame, delim=';')
new_column_names = names(Xdf_clean)

# Create an empty DataFrame to store the updated feature_partitions
col_names = Symbol.(names(feature_partitions))
col_types = eltype.(eachcol(feature_partitions))
named_tuple = (; zip(col_names, type[] for type in col_types )...)
updated_feature_partitions = DataFrame(named_tuple) 



# Iterate through the original features
for row in eachrow(feature_partitions)
    old_col = row[1]
    partition_values = row[2:end]
    
    # Check if the old feature exists in the new column names
    if old_col in new_column_names
        # If it exists, assign the partition values from the original CSV
        push!(updated_feature_partitions, (old_col, partition_values...))
    else
        # If it doesn't exist, it's a one-hot encoded feature
        # Find the corresponding one-hot encoded columns
        onehot_cols = filter(col -> startswith(col, old_col * "_"), new_column_names)
        
        # For each one-hot encoded column, assign the partition values from the original CSV
        for onehot_col in onehot_cols
            push!(updated_feature_partitions, (onehot_col, partition_values...), promote=true)
        end
    end
end


In [17]:

# Write the updated feature_partitions DataFrame to a new CSV file with the ";" delimiter
#CSV.write("updated_feature_partitions.csv", updated_feature_partitions, delim=';')
Pdf = updated_feature_partitions;

## Creating P and X matrix

In [18]:
P = Matrix(Pdf[!, 2:end])
X = Matrix(Xdf_clean[!, 1:end])
size(P), size(X)

((226, 10), (1460, 226))

# Training

## Parameters

In [19]:
η=10.0

10.0

In [22]:
y

1460-element Vector{Int64}:
 208500
 181500
 223500
 140000
 250000
 143000
 307000
 200000
 129900
 118000
      ⋮
 287090
 145000
  84500
 185000
 175000
 210000
 266500
 142125
 147500

## PartitionedLS model

In [24]:
model = fit(BnB, X, Vector{Float64}(y), P, η=η, nnlsalg=:nnls)

(opt = 1.2243547167598566e6,
 model = PartitionedLS.PartLSModel([0.0, 7.846207916172913e-6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005530551769617635, 0.0019167410854641779, 0.27623577952812534  …  0.0, 0.0, 0.3525103807101436, 0.0, 0.0, 0.0, 0.44438310523157376, 0.0, 0.0, 0.0], [50154.61798117696, -30843.64020322489, 30075.440878040077, 59173.71944354697, 22440.520705863444, -16105.461443353382, 16600.623773025713, -3330.1029537054624, 20659.401728566947, -25466.640056817687], -44.13943160818834, [1 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 1]),
 nopen = 325,)

## Regularized Least Squares model

In [25]:
Xo, Po = homogeneousCoords(X, P)

([65.0 8450.0 … 0.0 1.0; 80.0 9600.0 … 0.0 1.0; … ; 68.0 9717.0 … 0.0 1.0; 75.0 9937.0 … 0.0 1.0], [1 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 1 0; 0 0 … 0 1])

In [26]:
XX = Xo'Xo
wo = inv(XX + η*I)Xo' * y
lsq_w = wo[1:end-1]
lsq_t = wo[end]

-5.46533239049251

# Saving results

In [32]:
(; α,β,t,P) = model.model
alpha_df = DataFrame(colname=Pdf[!, 1], alpha = α[:,1])
beta_df = DataFrame(group=names(Pdf)[2:end], beta = β[:, 1])
t_df= DataFrame(t = t)

CSV.write("Results/PartLS_alpha.csv", alpha_df)
CSV.write("Results/PartLS_beta.csv", beta_df)
CSV.write("Results/PartLS_t.csv", t_df)

lsq_w_df = DataFrame(colname=Pdf[!, 1], lsq_w = lsq_w)
lsq_t_df = DataFrame(lsq_t = lsq_t)

CSV.write("Results/lsq_w.csv", lsq_w_df)
CSV.write("Results/lsq_t.csv", lsq_t_df)


"Results/lsq_t.csv"

# Analysis

**PartitionedLS accuracy**

In [34]:
mean(abs.(PartitionedLS.predict(model.model, X) - y))

18261.193582588734

**Least Squares accuracy**

In [35]:
mean(abs.(Xo * wo - y))

16698.55971058107

**Plotting**

In [37]:
# Pkg.add("PlotlyJS")
using PlotlyJS

In [39]:
betadf = DataFrame(groups=names(Pdf)[2:end], beta=vec(β));
alphadf = DataFrame(colname=Pdf[!, 1], alpha=vec(α));
lsq_w = DataFrame(colname=Pdf[!, 1], lsq_w=lsq_w);

In [41]:
layout = Layout(
    xaxis_title="Groups",
    yaxis_title="β",
    font_family="Computer Modern",
    font_size=16
)
# plot(betadf, layout, x=:groups, y=:beta, kind=:bar)
savefig(plot(betadf, layout, x=:groups, y=:beta, kind=:bar), "Figures/beta.pdf")

"Figures/beta.pdf"

In [42]:
layout = Layout(
    xaxis_title="Features",
    yaxis_title="w",
    xaxis_ticks=:all,
    font_family="Computer Modern",
    font_size=16
)
# savefig(plot(lsq_w, layout, x=:colname, y=:lsq_w, kind=:bar, xticks=:all), "Figures/lsq_w.pdf")
savefig(plot(lsq_w, layout, x=:colname, y=:lsq_w, kind=:bar, xticks=lsq_w.colname), "Figures/lsq_w.pdf")

"Figures/lsq_w.pdf"

In [43]:

layout = Layout(
    xaxis_title="Feature",
    yaxis_title="α",
    font_family="Computer Modern",
    font_size=16,
    rotation=45
)
for colName in names(Pdf)[2:end]
    coldf = alphadf[Pdf[!, Symbol(colName)] .== 1, :]
    savefig(plot(coldf, layout, x=:colname, y=:alpha, kind=:bar), "Figures/PartLS_alpha_$colName.pdf")
end
